In [ ]:
import math
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import tulipy as ti
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
import warnings
import os
import joblib

plt.style.use('seaborn-darkgrid')

In [ ]:
forex_data = yf.download('EURUSD=X', start='2019-01-02', end='2021-12-31')
forex_data.index = pd.to_datetime(forex_data.index)

In [ ]:
forex_data.head()

In [ ]:
forex_data.iloc[1:,:].head()

In [ ]:
# Plot the close price
plt.figure(figsize=(15, 7))
forex_data['Adj Close'].plot()

# Set the title and axis label
plt.title('EUR/USD Data', fontsize=16)
plt.xlabel('Year-Month', fontsize=15)
plt.ylabel('Price', fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(['Close'], prop={'size': 15})

# Show the plot
plt.show()

In [ ]:
# Set the ticker as 'EURUSD=X'
forex_data_minute = yf.download('EURUSD=X', period='7d', interval='1m')

# Set the index to a datetime object
forex_data_minute.index = pd.to_datetime(forex_data_minute.index)

# Display the last five rows
forex_data_minute.tail()

In [ ]:
def Labeling(data, windowSize):
    closePriceList = data['Adj Close'].values.tolist()
    counterRow, numberOfDaysInFile = 0, len(closePriceList)
    zeros_at_the_beginning = windowSize // 2
    labels = np.empty(zeros_at_the_beginning)
    labels.fill(np.nan)
    labels = labels.tolist()
    while counterRow < numberOfDaysInFile:
        min, max = math.inf, -math.inf
        counterRow += 1
        maxIndex, minIndex = -1, -1
        if counterRow >= windowSize:
            windowBeginIndex = counterRow - windowSize
            windowEndIndex = windowBeginIndex + windowSize - 1
            windowMiddleIndex = (windowBeginIndex + windowEndIndex) // 2
            for i in range(windowBeginIndex, windowEndIndex + 1):
                number = closePriceList[i]
                if number < min:
                    min = number
                    minIndex = i
                if number > max:
                    max = number
                    maxIndex = i
            if maxIndex == windowMiddleIndex:
                # labels.append("SELL")
                labels.append(2)
            elif minIndex == windowMiddleIndex:
                # labels.append("BUY")
                labels.append(1)
            else:
                # labels.append("HOLD")
                labels.append(0)

    zeros = np.empty(windowSize - zeros_at_the_beginning - 1)
    zeros.fill(np.nan)
    labels = np.append(labels, zeros.tolist())
    new_data = data.copy()
    new_data['Labels'] = labels
    return new_data

In [ ]:
forex_data_minute = Labeling(forex_data_minute, 5)
forex_data_minute.head(20)

In [ ]:
def ti_sma(period, data):
    res = ti.sma(data['Close'].values, period=period)
    newFrame = data.copy()
    newFrame[f'SMA_{period}'] = add_nans(period - 1, res)
    return newFrame


def ti_ema(period, data):
    res = ti.ema(data['Close'].values, period=period)
    newFrame = data.copy()
    newFrame[f'EMA_{period}'] = res
    return newFrame


def ti_hma(period, data):
    res = ti.hma(data['Close'].values, period=period)
    if period <= 8:
        change = 0
    elif period <= 15:
        change = 1
    elif period <= 24:
        change = 2
    else:
        change = 3
    newFrame = data.copy()
    newFrame[f'HMA_{period}'] = add_nans(period + change, res)
    return newFrame


def ti_wma(period, data):
    res = ti.wma(data['Close'].values, period=period)
    newFrame = data.copy()
    newFrame[f'WMA_{period}'] = add_nans(period - 1, res)
    return newFrame


def ti_triple_ema(period, data):
    res = ti.tema(data['Close'].values, period=period)
    newFrame = data.copy()
    newFrame[f'T_EMA_{period}'] = add_nans(period * 3 - 3, res)
    return newFrame


def add_nans(period, res):
    a = np.empty(period)
    a.fill(np.nan)
    return np.append(a, res)


def ti_willr(period, data):
    res = ti.willr(data['High'].values, data['Low'].values, data['Close'].values, period=period)
    newFrame = data.copy()
    newFrame[f'WILLR_{period}'] = add_nans(period - 1, res)
    return newFrame


def ti_rsi(period, data):
    res = ti.rsi(data['Close'].values, period=period)
    newFrame = data.copy()
    newFrame[f'RSI_{period}'] = add_nans(period, res)
    return newFrame


def ti_cci(period, data):
    res = ti.cci(data['High'].values, data['Low'].values, data['Close'].values, period=period)
    newFrame = data.copy()
    newFrame[f'CCI_{period}'] = add_nans(period + period - 2, res)
    return newFrame


def ti_cmo(period, data):
    res = ti.cmo(data['Close'].values, period=period)
    newFrame = data.copy()
    newFrame[f'CMO_{period}'] = add_nans(period, res)
    return newFrame


def ti_roc(period, data):
    res = ti.roc(data['Close'].values, period=period)
    newFrame = data.copy()
    newFrame[f'ROC_{period}'] = add_nans(period, res)
    return newFrame


# extra indicator for firex data to make the image of size 15x15
def ti_kama(period, data):
    res = ti.kama(data['Close'].values, period=period)
    newFrame = data.copy()
    newFrame[f'KAMA_{period}'] = add_nans(period - 1, res)
    return newFrame


def ti_dmi(period, data):
    res = ti.dx(data['High'].values, data['Low'].values, data['Close'].values, period=period)
    newFrame = data.copy()
    newFrame[f'DMI_{period}'] = add_nans(period - 1, res)
    return newFrame


def get_periods(period):
    return period, period * 2, int(0.75 * period)


def ti_macd(period, data):
    shortTermEma = ti.ema(data['Close'].values, period=period)
    longTermEma = ti.ema(data['Close'].values, period=period * 2)
    res = shortTermEma - longTermEma
    newFrame = data.copy()
    newFrame[f'MACD_{period}'] = res
    return newFrame


def ti_ppo(period, data):
    short_period, long_period, _ = get_periods(period)
    res = ti.ppo(data['Close'].values, short_period, long_period)
    newFrame = data.copy()
    newFrame[f'PPO_{period}'] = add_nans(1, res)
    return newFrame


# etfs have volumes, forex data doesn't, for the forex data we use kama indicator instead
def ti_cmfi(period, data):
    res = ti.mfi(data['High'].values, data['Low'].values, data['Close'].values, data['Volume'].values, period=period)
    newFrame = data.copy()
    newFrame[f'CMFI_{period}'] = add_nans(period, res)
    return newFrame


def ti_psar(period, data):
    res, data_high, data_low = [np.NAN], data['High'].values, data['Low'].values
    for i in range(1, len(data)):
        start = max(0, i - period + 1)
        res.append(ti.psar(data_high[start:i + 1], data_low[start:i + 1], 0.02, 0.2)[-1])
    newFrame = data.copy()
    newFrame[f'PSAR_{period}'] = res
    return newFrame

In [ ]:
forex_data_minute = ti_psar(5, forex_data_minute)
forex_data_minute

In [ ]:
# there is no volume records for forex data, so we skip this step
def adjust_the_prices(data):
    data['Open'] = (data['Open'] * data['Volume']) / data['Adj Close']
    data['High'] = (data['High'] * data['Volume']) / data['Adj Close']
    data['Low'] = (data['Low'] * data['Volume']) / data['Adj Close']
    # data.rename(columns={"Adj Close": "Close"}, inplace=True)
    return data

In [ ]:
def read_data(directory, ticket, forex):
    datas = []
    for file in sorted(os.listdir(directory)):
        if ticket in file:
            data = pd.read_csv(os.path.join(directory, file), index_col=['Datetime'])
            data.index = pd.to_datetime(data.index)
            print('read file - ', file, 'len = ', len(data))
            print('len data before = ', len(data))
            data = create_data(data, forex)
            print('len data after2 = ', len(data))
            datas.append(data)
    data = datas[0]
    for i in range(1, len(datas)):
        data = data.append(datas[i])
    return data

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)


def create_data(data, forex=True):
    if not forex:
        data = adjust_the_prices(data)

    # label the data: BUY(1), SELL(2), HOLD(0)
    data = data.astype('float64')
    data = Labeling(data, 11)

    # order of indicators matters
    eft_indicators = [ti_rsi, ti_willr, ti_wma, ti_ema, ti_sma, ti_hma, ti_triple_ema, ti_cci, ti_cmo, ti_macd, ti_ppo,
                      ti_roc, ti_cmfi, ti_dmi, ti_psar]

    forex_indicators = [ti_rsi, ti_willr, ti_wma, ti_ema, ti_sma, ti_kama, ti_hma, ti_triple_ema, ti_cci, ti_cmo,
                        ti_macd, ti_ppo, ti_roc, ti_dmi, ti_psar]

    for period in range(5, 20):
        if forex:
            indicators = forex_indicators
        else:
            indicators = eft_indicators
        for indicator in indicators:
            data = indicator(period, data)

    # do not remove 'Adj Close'
    data.drop(labels=["Volume", "Low", "High", "Open", "Close"], axis=1, inplace=True)
    data.dropna(inplace=True)

    # TODO: round TA indicators up to 2 values after the point after normalization?? (according to the article)
    return data

In [ ]:
# data = read_data('./historical_data/forex_minutes/', 'EURUSD=X', forex=True)
data = read_data('./historical_data/etfs_minutes/', 'EWH', forex=False)


In [ ]:
data

In [ ]:
def save_to_csv():
    start = ['2023-08-20', '2023-08-27', '2023-09-03', '2023-09-10']
    end = ['2023-08-27', '2023-09-03', '2023-09-10', '2023-09-17']
    currencies = ['EURUSD=X', 'EURTRY=X', 'EURRUB=X', 'ARSUSD=X', 'AUDUSD=X', 'JPYUSD=X', 'IRRUSD=X', 'KPW=X',
                  'EURDKK=X', 'SEKUSD=X']
    etfs = ['XLF', 'QQQ', 'SPY', 'XLP', 'EWZ', 'EWH', 'XLY', 'XLE']
    files = os.listdir('./historical_data/forex_minutes') + (os.listdir('./historical_data/etfs_minutes'))

    for i in range(4):
        for j in range(len(currencies)):
            file_name = f'{start[i]}_to_{end[i]}_{currencies[j]}.csv'
            if file_name not in files:
                data = yf.download(currencies[j], start=start[i], end=end[i], interval='1m')
                if len(data) > 0:
                    data.to_csv('historical_data/forex_minutes/' + file_name)
            else:
                print('already exists: ', file_name)
        for j in range(len(etfs)):
            file_name = f'{start[i]}_to_{end[i]}_{etfs[j]}.csv'
            if file_name not in files:
                data = yf.download(etfs[j], start=start[i], end=end[i], interval='1m')
                if len(data) > 0:
                    data.to_csv('historical_data/etfs_minutes/' + file_name)
            else:
                print('already exists: ', file_name)


In [ ]:
save_to_csv()

In [ ]:
import joblib
def normalize(data):
    scaler = MinMaxScaler((-1, 1))
    data_to_normalize = data.drop(labels=["Labels", "Adj Close"], axis=1)
    columns = data_to_normalize.columns
    data_scaled = scaler.fit_transform(data_to_normalize.to_numpy())
    joblib.dump(scaler, 'prediction_tools/TCSG_minutes_scaler.sav')
    # index gets removed, columns are also not needed, might be removed too
    data_scaled = pd.DataFrame(data_scaled, columns=columns)
    data_scaled['Adj Close'] = data['Adj Close'].values
    data_scaled['Labels'] = data['Labels'].values
    return data_scaled

In [ ]:
normalized_data = normalize(data)
normalized_data

In [ ]:
def test_loaded_scaler(data):
    scaler = joblib.load('prediction_tools/TCSG_minutes_scaler_testing.sav')
    data_to_normalize = data.drop(labels=["Labels", "Adj Close"], axis=1)
    columns = data_to_normalize.columns
    data_scaled = scaler.fit_transform(data_to_normalize.to_numpy())
    data_scaled = pd.DataFrame(data_scaled, columns=columns)
    data_scaled['Adj Close'] = data['Adj Close'].values
    data_scaled['Labels'] = data['Labels'].values
    return data_scaled

In [ ]:
normalized_data = test_loaded_scaler(data)
normalized_data.tail()

In [ ]:
# TODO: split into training and test sets
# 4 weeks altogether, let's split into 3/1

training_proportion = int(0.75 * len(normalized_data))
training_set = normalized_data[:training_proportion]
testing_set = normalized_data[training_proportion:]

In [ ]:
# Suppress FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)


def solve_imbalance_problem(data):
    data = data.iloc[15:, :]

    l0_train = data.loc[data['Labels'] == 0]
    l1_train = data.loc[data['Labels'] == 1]
    l2_train = data.loc[data['Labels'] == 2]
    l0_size = l0_train.shape[0]
    l1_size = l1_train.shape[0]
    l2_size = l2_train.shape[0]

    l0_l1_ratio = (l0_size // l1_size)
    l0_l2_ratio = (l0_size // l2_size)
    print("Before")
    print("l0_size:", l0_size, "l1_size:", l1_size, "l2_size:", l2_size)
    print("l0_l1_ratio:", l0_l1_ratio, "l0_l2_ratio:", l0_l2_ratio)

    l1_new = pd.DataFrame()
    l2_new = pd.DataFrame()
    for idx, row in data.iterrows():
        if row['Labels'] == 1:
            for i in range(l0_l1_ratio):
                l1_new = l1_new.append(row)
        if row['Labels'] == 2:
            for i in range(l0_l2_ratio):
                l2_new = l2_new.append(row)

    data = data.append(l1_new)
    data = data.append(l2_new)

    # shuffle
    data = shuffle(data)

    ########################################################
    l0_train = data.loc[data['Labels'] == 0]
    l1_train = data.loc[data['Labels'] == 1]
    l2_train = data.loc[data['Labels'] == 2]
    l0_size = l0_train.shape[0]
    l1_size = l1_train.shape[0]
    l2_size = l2_train.shape[0]

    l0_l1_ratio = (l0_size // l1_size)
    l0_l2_ratio = (l0_size // l2_size)
    print("After")
    print("l0_size:", l0_size, "l1_size:", l1_size, "l2_size:", l2_size)
    print("l0_l1_ratio:", l0_l1_ratio, "l0_l2_ratio:", l0_l2_ratio)

    return data

In [ ]:
# only for training set
training_set = solve_imbalance_problem(training_set)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import shuffle
import math


def reverse_one_hot(predictions):
    reversed_x = []
    for x in predictions:
        reversed_x.append(np.argmax(np.array(x)))
    return reversed_x


def train_cnn(training_df, test_df, params):
    """Trains and evaluates CNN on the given train and test data, respectively."""

    print("Training is starting ...")
    train_images = np.array(training_df.iloc[:, :-2].values.tolist())
    train_labels = training_df['Labels']
    train_prices = training_df['Adj Close']

    test_images = np.array((test_df.iloc[:, :-2].values.tolist()))
    test_labels = test_df['Labels']
    test_prices = test_df['Adj Close']

    test_labels = keras.utils.to_categorical(test_labels, params["num_classes"])
    train_labels = keras.utils.to_categorical(train_labels, params["num_classes"])

    train_images = train_images.reshape(train_images.shape[0], params["input_w"], params["input_h"], 1)
    test_images = test_images.reshape(test_images.shape[0], params["input_w"], params["input_h"], 1)

    # CNN model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(params["input_w"], params["input_h"], 1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(params["num_classes"], activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy', 'mae', 'mse'])

    train_data_size = train_images.shape[0]
    test_data_size = test_images.shape[0]

    print("model will be trained with {} and be tested with {} sample".format(train_data_size, test_data_size))
    # fit the model to the training data
    print("Fitting model to the training data...")
    print("")
    model.fit(train_images, train_labels, batch_size=params["batch_size"], epochs=params["epochs"], verbose=1,
              validation_data=None)

    predictions = model.predict(test_images, batch_size=params["batch_size"], verbose=1)
    print(model.evaluate(test_images, test_labels, batch_size=params["batch_size"], verbose=1))

    print("Train conf matrix: ", confusion_matrix(np.array(reverse_one_hot(train_labels)),
                                                  np.array(reverse_one_hot(
                                                      model.predict(train_images, batch_size=params["batch_size"],
                                                                    verbose=1)))))

    print("Test conf matrix: ", confusion_matrix(np.array(reverse_one_hot(test_labels)),
                                                 np.array(reverse_one_hot(predictions))))

    return predictions, test_labels, test_prices

In [ ]:
print("train_df size: ", training_set.shape)

# TODO: should round values to two values after point??

# fill params dict before call train_cnn
params = {"input_w": 15, "input_h": 15, "num_classes": 3, "batch_size": 1024, "epochs": 200}
#params = {"input_w": 15, "input_h": 15, "num_classes": 3, "batch_size": 1024, "epochs": 100}
training_set.reset_index(drop=True, inplace=True)
testing_set.reset_index(drop=True, inplace=True)

predictions, test_labels, test_prices = train_cnn(training_set, testing_set, params)

result_df = pd.DataFrame({"prediction": np.argmax(predictions, axis=1),
                          "test_label": np.argmax(test_labels, axis=1),
                          "test_price": test_prices})


In [ ]:
result_df

In [ ]:
print(classification_report(result_df['test_label'], result_df['prediction']))

In [ ]:
print(classification_report(result_df['test_label'], result_df['prediction']))

Precision means the percentage of correct positive predictions relative to total positive predictions. Recall reflects percentage of correct positive predictions relative to total actual positives. F1 Score represents weighted harmonic mean of precision and recall. The closer it is to 1, the better the model.

In [ ]:
# 10000 - starting capital in certain currency
def calculate_profit(money, predictions, prices):
    # TODO: have to consider transaction fee and number of transactions
    transactions_count = 0
    last_sell = len(predictions) - predictions[::-1].index(2) - 1  # find last Sell label
    first_buy = predictions.index(1)  #find first buy
    predictions = predictions[first_buy:last_sell + 1]
    prices = prices[first_buy:last_sell + 1]
    # print(len(predictions), len(prices))
    ofCurrency = 0
    currentLabel = 2
    for i in range(len(predictions)):
        label = predictions[i]
        # print('found label:', label)
        if label == 1:
            if currentLabel != 1:
                currentLabel = 1
                # print('change in 1: before: ', ofCurrency, money, prices[i])
                transactions_count += 1
                ofCurrency = money / prices[i]
                money = 0
                # print('change in 1: after: ', ofCurrency, money)
        elif label == 2:
            if currentLabel != 2:
                currentLabel = 2
                # print('change in 2: before: ', ofCurrency, money, prices[i])
                transactions_count += 1
                money = prices[i] * ofCurrency
                ofCurrency = 0
                # print('change in 2: after: ', ofCurrency, money)            
    return money, ofCurrency, transactions_count

In [ ]:
predictions = result_df['prediction'].tolist()
startMoney = 10000
totalMoney, ofCurrency, transactionCount = calculate_profit(startMoney, predictions, result_df['test_price'])

In [ ]:
# if constant_transaction_fee is False, fee is a percent of the traded money (fee=0.05 for example)
# if constant_transaction_fee is True, fee is a constant number of units in certain currency (like 1 euro) 
def strategy(labels, prices, fee, constant_transaction_fee=True):
    i, totalTransactionLength = 0, 0
    buyPoint, sellPoint, gain, totalGain, shareNumber, moneyTemp, maximumMoney = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
    money, minimumMoney = 10000.0, 10000.0
    maximumLost = 100.0
    totalPercentProfit, maximumProfitPercent, maximumLostPercent, maximumGain = 0.0, 0.0, 0.0, 0.0
    transactionCount, successTransactionCount, failedTransactionCount = 0, 0, 0
    buyPointBAH, shareNumberBAH, moneyBAH = 10000.0, 10000.0, 10000.0
    forceSell = False

    sharpaR, prof, oneDayProf, numberOfDay, k = 0, 0, 0, 0, 0

    # TODO: define non-constant transaction fee (for example a percent of the whole transaction)

    print(f"Start Capital: {money}$")
    while k < len(labels) - 1:

        # dailyProfit[k] = 0.0

        if labels[k] == 1.0:
            buyPoint = prices[k]
            buyPoint = buyPoint * 100
            if constant_transaction_fee:
                shareNumber = (money - fee) / buyPoint
            else:
                shareNumber = (money - money * fee) / buyPoint
            forceSell = False

            for j in range(k, len(labels) - 1):
                sellPoint = prices[j]
                sellPoint = sellPoint * 100
                if constant_transaction_fee:
                    moneyTemp = (shareNumber * sellPoint) - fee
                else:
                    help = shareNumber * sellPoint
                    moneyTemp = help - help * fee
                # stop loss %10
                # if(money*0.95>moneyTemp){
                # 	money=moneyTemp;
                # 	forceSell=true;
                # }
                if labels[j] == 2.0 or forceSell:
                    sellPoint = prices[j]
                    sellPoint = sellPoint * 100
                    gain = sellPoint - buyPoint
                    if gain > 0:
                        successTransactionCount += 1
                    else:
                        failedTransactionCount += 1
                    if gain >= maximumGain:
                        maximumGain = gain
                        maximumProfitPercent = maximumGain / buyPoint * 100
                    if gain <= maximumLost:
                        maximumLost = gain
                        maximumLostPercent = maximumLost / buyPoint * 100
                    if constant_transaction_fee:
                        moneyTemp = (shareNumber * sellPoint) - fee
                    else:
                        help = shareNumber * sellPoint
                        moneyTemp = help - help * fee
                    money = moneyTemp
                    if money > maximumMoney:
                        maximumMoney = money
                    if money < minimumMoney:
                        minimumMoney = money
                    transactionCount += 1
                    print(f'{transactionCount}.({k + 1}-{j + 1}) => {round((gain * shareNumber), 2)} Capital: {round(money, 2)}$')
                    prof = round(round((gain * shareNumber), 2) / (money - (gain * shareNumber)), 4)
                    numberOfDay = j - k
                    # oneDayProf = round((prof / numberOfDay), 4)
                    # for m in range(k + 1, j + 1):
                    #     dailyProfit[m] = oneDayProf

                    totalPercentProfit = totalPercentProfit + (gain / buyPoint)

                    totalTransactionLength = totalTransactionLength + (j - k)
                    k = j + 1
                    totalGain = totalGain + gain
                    break
        k += 1

    # print("dailyProfit[z]")
    # for z in range(0, dailyProfit.length):
    #     print(f'{z}:{dailyProfit[z]}')
    # sharpaR = findSharpaRatio(dailyProfit)
    # 
    # print("Sharpa Ratio of Our System=>" + sharpaR)
    print(f"Our System => totalMoney = {round(money, 2)}$")

    buyPointBAH = prices[0]
    if constant_transaction_fee:
        shareNumberBAH = (moneyBAH - fee) / buyPointBAH
        moneyBAH = (prices[len(labels) - 1] * shareNumberBAH) - fee
    else:
        shareNumberBAH = (moneyBAH - moneyBAH * fee) / buyPointBAH
        help = prices[len(labels) - 1] * shareNumberBAH
        moneyBAH = help - help * fee
        
    print(f'BAH => totalMoney = {round(moneyBAH, 2)}$')

    numberOfDays = len(labels) - 1
    numberOfYears = numberOfDays / 365

    print(f"Our System Annualized return % => {round(((math.pow(money / 10000.0, 0.2) - 1) * 100), 2)}%")  #5 years 0.2
    print(
        f"BaH Annualized return % => {round(((math.exp(math.log(moneyBAH / 10000.0)) - 1) * 100), 2)} %")
    print(f"Annualized number of transaction => {round((transactionCount), 1)} #")
    print(f"Percent success of transaction => {round((successTransactionCount / transactionCount) * 100, 2)} %")
    print(f"Average percent profit per transaction => {round((totalPercentProfit / transactionCount * 100), 2)} %")
    print(f"Average transaction length => {totalTransactionLength / transactionCount} #")
    print(f"Maximum profit percent in transaction=> {round(maximumProfitPercent, 2)} %")
    print(f"Maximum loss percent in transaction=> {round(maximumLostPercent, 2)} %")
    print(f"Maximum capital value=> {round(maximumMoney, 2)}$")
    print(f"Minimum capital value=> {round(minimumMoney, 2)}$")
    print(f"Idle Ratio %=> {round((len(labels) - totalTransactionLength / len(labels) * 100), 2)} %")

In [ ]:
labels = result_df['prediction'].tolist()
prices = result_df['test_price'].tolist()
strategy(labels, prices, 0.5, constant_transaction_fee=True)

In [ ]:
strategy(labels, prices, 0.005, constant_transaction_fee=False)

In [ ]:
labels = result_df['prediction'].tolist()
prices = result_df['test_price'].tolist()
strategy(labels, prices, 0.5, constant_transaction_fee=True)

In [ ]:
strategy(labels, prices, 0.005, constant_transaction_fee=False)


In [ ]:
d = '11111#2222#33333#44444'
position = d[::-1].find('#')
new_string = d[::-1][position+1:][::-1]
new_string

In [ ]:
import numpy as np
s = np.array([1,2,3])
s = np.append(s, 4)
s

In [ ]:
x = [[1,2,3],[4,5,6]]
x_new = [[7,8,9],[10,11,12]]
for xx in x_new:
    x.append(xx)
print(x)

In [ ]:
import tulipy as ti
import numpy as np
r = np.array([3249.5, 3249.5,3249.5 ,3249.5, 3249.5, 3249.5, 3249.5, 3249.5, 3249.5, 3249.5,
 3249.5, 3249.5, 3249.5 ,3249.5, 3249.5, 3249.5])
res = ti.tema(r, period=7)
print(res)

In [ ]:
df_1 = pd.read_csv('prediction_tools/data_to_normalize.csv', index_col=['Datetime'])
df_1.head()

In [ ]:
normalized_data = test_loaded_scaler(df_1)
normalized_data

In [ ]:
# normalized_data = test_loaded_scaler(data)
# normalized_data
data.head()

In [ ]:
def get_best_model_params(params):
    params = sorted(params, key=lambda x: x[1][1][2]+x[1][2][1]) 
    i = 1
    min_false_negatives_num = params[0][1][1][2]+params[0][1][2][1]
    while params[i][1][1][2]+params[i][1][2][1] == min_false_negatives_num:
        i += 1
    params = sorted(params[:i], key=lambda x: -x[0]) 
    return params[0][2], params[0][3], params[0][4]

In [ ]:
conf_mat1 = [[354, 225, 261],
 [134, 699, 4],
 [142, 3, 695]]

conf_mat2 = [[354, 225, 261],
 [134, 699, 6],
 [142, 3, 695]]

conf_mat3 = [[354, 225, 261],
 [134, 699, 5],
 [142, 3, 695]]

conf_mat4 = [[354, 225, 261],
 [134, 699, 4],
 [142, 3, 695]]

params = [(0.98, conf_mat1, 1, 2, 3), (0.99, conf_mat2, 0, 0, 0), (0.99, conf_mat3, 0, 0, 0), (0.97, conf_mat4, 4, 5, 6)]
sorrrrted = get_best_model_params(params)
print(sorrrrted)

In [ ]:
def normalize(data, filename):
    scaler = MinMaxScaler((-1, 1))
    data_to_normalize = data.drop(labels=["Labels", "Adj Close"], axis=1)
    columns = data_to_normalize.columns

    # -212.65458552306788 109066.66666666667 (17221, 225) - min, max, shape
    # data_to_normalize.to_csv('prediction_tools/data_to_normalize_whole.csv')

    data_scaled = scaler.fit_transform(data_to_normalize.to_numpy())
    joblib.dump(scaler, filename)
    # index gets removed, columns are also not needed, might be removed too
    data_scaled = pd.DataFrame(data_scaled, columns=columns)
    data_scaled['Adj Close'] = data['Adj Close'].values
    data_scaled['Labels'] = data['Labels'].values
    return data_scaled

def read_data_from_single_file(file_name, forex):
    data = pd.read_csv(file_name, index_col=['Datetime'])
    data.index = pd.to_datetime(data.index)
    return create_data(data, forex)

data = read_data_from_single_file("./historical_data/TCSG_minutes/2023-10-13_to_2023-11-12_TCSG.csv", forex=False)

normalized_data = normalize(data, 'prediction_tools/TCSG_minutes_scaler.sav')

training_proportion = int(0.7 * len(normalized_data))
training_set = normalized_data[:training_proportion]
testing_set = normalized_data[training_proportion:]


In [ ]:
training_set.head()

In [ ]:
from datetime import datetime
print(datetime.now().strftime("%d/%m/%Y %H:%M:%S"))